In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pickle

import warnings
warnings.filterwarnings('ignore')

import tidalUtl.PrpUtl as prp
import tidalUtl.EdaUtl as eda

In [2]:
print(torch.__version__) 

1.7.0+cu110


# Version

__ver1__<br>
baseline：CV:0.01465 LB:0.01874<br>
__ver2__<br>
Hyperopt, 2Layer：CV:0.01460 LB:0.01869<br>
__ver3__<br>
3Layer：CV:0.01464 LB:0.01868<br>
__ver4__<br>
MLSMOTE baseline：CV:0.01476 LB:0.01978<br>
__ver5__<br>
2Layer,refactoring：CV:0.01476 LB:0.01869<br>
__ver6__<br>
rankGauss：CV:0.01456 LB:0.01865<br>
__ver7__<br>
labelSmoothing：CV:0.01502 LB:0.01859<br>

# Config

In [3]:
INPUT = "/home/tidal/ML_Data/CIFAR10/cifar-10-python/cifar-10-batches-py"
OUTPUT = "/home/tidal/ML_Data/CIFAR10/output"

SUBMIT = OUTPUT + "/submittion/"
SAVEMODEL = OUTPUT + "/model/Pytorch/"
SAVEOOF = OUTPUT + "/OOF/Pytorch/"

In [4]:
#Loading
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

metaData = unpickle(INPUT + '/batches.meta')
batch1 = unpickle(INPUT + '/data_batch_1')
batch2 = unpickle(INPUT + '/data_batch_2')
batch3 = unpickle(INPUT + '/data_batch_3')
batch4 = unpickle(INPUT + '/data_batch_4')
batch5 = unpickle(INPUT + '/data_batch_5')
testBatch = unpickle(INPUT + '/test_batch')

#trainは分割されていたのを一つに結合(この時点ではまだseries)
trainFeature = np.concatenate([batch1[b'data'], 
                               batch2[b'data'],
                               batch3[b'data'],
                               batch4[b'data'],
                               batch5[b'data'],])
testFeature = testBatch[b'data']

trainTarget = np.concatenate([batch1[b'labels'], 
                               batch2[b'labels'],
                               batch3[b'labels'],
                               batch4[b'labels'],
                               batch5[b'labels'],])
testTarget = np.array(testBatch[b'labels'])


#pandasとして扱う。
trainFeature = pd.DataFrame(trainFeature)
trainTarget = pd.Series(trainTarget)
testFeature = pd.DataFrame(testFeature)
testTarget = pd.Series(testTarget)

#targetのカラム
tarClmn = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [5]:
#Seed固定
def seed_everything(seed=42):
    #data取得についてのランダム性固定
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #cudnnによる演算の安定化(評価値の安定)
    torch.backends.cudnn.deterministic = True
    
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(seed=42)

In [6]:
#HyperParameter
param_space = {'hidden_size1': 512, 
               'hidden_size2': 512, 
               'dropOutRate1': 0.20393004966355735, 
               'dropOutRate2': 0.39170486751620137,
               'leakyReluSlope': 0.01973893854348531,
              }

# Preprocessing

## Func: In & Out Type is DataFrame

## Collecting

__train,testにターゲット値も連結__

In [7]:
def Collecting(trainFeature, testFeature, trainTarget):
    #targetを結合
    trainFeature = pd.concat([trainFeature, trainTarget], axis=1)
    #test側のtargetは0で初期化しておく
    testTarTmp = pd.DataFrame(np.zeros((testFeature.shape[0],trainTarget.shape[1]),dtype='uint8'), columns=tarClmn)
    testFeature = pd.concat([testFeature, testTarTmp], axis=1)
    
    
    return trainFeature, testFeature

## Preprocessing Summary

In [8]:
def preprocessing(param, trainFeature, testFeature, trainTarget):
    
    #targetのデータフレーム作成（列名も付与）(one-hot化しておく)
    targetOH = pd.get_dummies(trainTarget)
    targetOH.columns = tarClmn
    
    #train,testにターゲット値を連結。
    train, test = Collecting(trainFeature, testFeature, targetOH)
    
    return train, test, targetOH

## Work

## Visualization

In [9]:
%%time
trainVsl, testVsl, targetVsl = preprocessing(param_space, trainFeature, testFeature, trainTarget)

CPU times: user 966 ms, sys: 7.08 ms, total: 973 ms
Wall time: 973 ms


In [10]:
trainVsl.head(5)

,0,1,2,3,4,5,6,7,8,9,...,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,59,43,50,68,98,119,139,145,149,149,...,0,0,0,0,0,0,1,0,0,0
1,154,126,105,102,125,155,172,180,142,111,...,0,0,0,0,0,0,0,0,0,1
2,255,253,253,253,253,253,253,253,253,253,...,0,0,0,0,0,0,0,0,0,1
3,28,37,38,42,44,40,40,24,32,43,...,0,0,0,0,1,0,0,0,0,0
4,170,168,177,183,181,177,181,184,189,189,...,0,1,0,0,0,0,0,0,0,0


In [11]:
testVsl.head(5)

,0,1,2,3,4,5,6,7,8,9,...,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,158,159,165,166,160,156,162,159,158,159,...,0,0,0,0,0,0,0,0,0,0
1,235,231,232,232,232,232,232,232,232,232,...,0,0,0,0,0,0,0,0,0,0
2,158,158,139,132,166,182,187,193,199,205,...,0,0,0,0,0,0,0,0,0,0
3,155,167,176,190,177,166,168,166,170,179,...,0,0,0,0,0,0,0,0,0,0
4,65,70,48,30,23,40,44,45,45,40,...,0,0,0,0,0,0,0,0,0,0


In [12]:
targetVsl.head(5)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0


In [13]:
print("Train: "+ str(trainVsl.shape))
print("Test: "+ str(testVsl.shape))
print("Target: "+ str(targetVsl.shape))

Train: (50000, 3082)
Test: (10000, 3082)
Target: (50000, 10)


# Fitting

## Config about Fitting

In [14]:
#configは辞書化しておく。
def Config_about_Fitting(train, test, target, folds):
    confFitting = {}
    
    #Fitするときに"y"として使う列の列名配列
    confFitting["target_cols"] = target.columns.values.tolist()
    #Fitするときに"X"として使う列の列名配列
    #kfold, id等はここで削除。
    feature_cols = [c for c in folds.columns if c not in confFitting["target_cols"]]
    confFitting["feature_cols"] = [c for c in feature_cols if c not in ['kfold','sig_id']]
    #特徴量、ターゲットのサイズ
    confFitting["num_features"]=len(confFitting["feature_cols"])
    confFitting["num_targets"]=len(confFitting["target_cols"])
    
    return confFitting

## Dataset Classes

In [15]:
#Train,Valid用のデータクラス
class TrainDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        #torch.DataLoaderに入れるための形式
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
#Test用のデータクラス
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            #torch.DataLoaderに入れるための形式
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

## Loss, Metric

In [16]:
#loss
#nn.BCEWithLogitsLoss()

#class LabelSmoothingCrossEntropy(nn.Module):
#    def __init__(self):
#        super(LabelSmoothingCrossEntropy, self).__init__()
#    def forward(self, x, target, smoothing=0.001):
#        confidence = 1. - smoothing
#        logprobs = F.log_softmax(x, dim=-1)
#        bcs_loss = nn.BCEWithLogitsLoss()(x, target)
#        smooth_loss = -logprobs.mean(dim=-1)
#        loss = confidence * bcs_loss + smoothing * smooth_loss
#        return loss.mean()

In [17]:
#metric
#nn.BCEWithLogitsLoss()

## Func: Fitting, Evaluation, Predict

In [18]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

## Model architect

In [19]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, param):
        super(Model, self).__init__()
        #hyperoptによる被探索パラメータ
        hidden_size1=param['hidden_size1']
        hidden_size2=param['hidden_size2']
        dropOutRate1=param['dropOutRate1']
        dropOutRate2=param['dropOutRate2']
        leakyReluSlope=param['leakyReluSlope']
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(dropOutRate1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size1))
        self.leakyRelu1 = nn.LeakyReLU(negative_slope=leakyReluSlope)
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size1)
        self.dropout2 = nn.Dropout(dropOutRate2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size1, hidden_size2))
        self.leakyRelu2 = nn.LeakyReLU(negative_slope=leakyReluSlope)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size2)
        self.dropout3 = nn.Dropout(dropOutRate2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size2, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = self.leakyRelu1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.leakyRelu2(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

# Run

## HyperParameter

In [20]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 512
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

## CV folds

In [21]:
trainTarget

0        6
1        9
2        9
3        4
4        1
        ..
49995    2
49996    6
49997    9
49998    1
49999    1
Length: 50000, dtype: int64

In [22]:
def CV_folds(train, target):
    folds = train.copy()
    
    skf = StratifiedKFold(n_splits=NFOLDS)
    
    for f, (t_idx, v_idx) in enumerate(skf.split(X=train, y=trainTarget)):
        folds.loc[v_idx, 'kfold'] = int(f)
    
    folds['kfold'] = folds['kfold'].astype(int)
    
    return folds

In [23]:
%%time
#Preprocessing Data
trainVsl, testVsl, targetVsl = preprocessing(param_space, trainFeature, testFeature, trainTarget)
#CV folds
foldsVsl = CV_folds(trainVsl, targetVsl)

foldsVsl.head(5)

CPU times: user 1.03 s, sys: 31.5 ms, total: 1.06 s
Wall time: 1.06 s


,0,1,2,3,4,5,6,7,8,9,...,automobile,bird,cat,deer,dog,frog,horse,ship,truck,kfold
0,59,43,50,68,98,119,139,145,149,149,...,0,0,0,0,0,1,0,0,0,0
1,154,126,105,102,125,155,172,180,142,111,...,0,0,0,0,0,0,0,0,1,0
2,255,253,253,253,253,253,253,253,253,253,...,0,0,0,0,0,0,0,0,1,0
3,28,37,38,42,44,40,40,24,32,43,...,0,0,0,1,0,0,0,0,0,0
4,170,168,177,183,181,177,181,184,189,189,...,1,0,0,0,0,0,0,0,0,0


In [24]:
foldsVsl.kfold.value_counts()

4    10000
3    10000
2    10000
1    10000
0    10000
Name: kfold, dtype: int64

## Single Fold Running

In [25]:
def run_training(confFitting, Tester, fold, seed, param,
                 folds, train, test, target):
    
    seed_everything(seed)
    
    train = folds
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[confFitting["feature_cols"]].values, train_df[confFitting["target_cols"]].values
    x_valid, y_valid =  valid_df[confFitting["feature_cols"]].values, valid_df[confFitting["target_cols"]].values
    
    train_dataset = TrainDataset(x_train, y_train)
    valid_dataset = TrainDataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    ##### 評価関数 ######
    train_loss_fn = nn.BCEWithLogitsLoss()
    valid_loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, train_loss_fn, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, valid_loss_fn, validloader, DEVICE)
        if Tester:
            print("EPOCH: {:03}: | train_loss: {:.3f}: | valid_loss: {:.3f}".format(epoch, train_loss, valid_loss))
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth")
        
        elif(EARLY_STOP == True):
            early_step += 1
            if (early_step >= early_stopping_steps):
                if Tester:
                    print('Early stopping. Best Val loss: {:.3f}'.format(best_loss))
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test[confFitting["feature_cols"]].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.load_state_dict(torch.load(f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    
    return oof, predictions


## K-Fold Running

In [26]:
def run_k_fold(Tester, NFOLDS, seed, param,
              folds, train, test, target, confFitting):
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        oof_, pred_ = run_training(confFitting, Tester, fold, seed, param,
                                   folds, train, test, target)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

## CV Evaluation

In [27]:
 def CV_Evaluation(confFitting, oof, target):
    
    y_true = target[confFitting["target_cols"]].values
    y_pred = oof
    
    score = 0
    for i in range(confFitting["num_targets"]):
        score_ = log_loss(y_true[:, i], y_pred[:, i]) #問題の評価指標によって変わる。
        score += score_ / target.shape[1]
        
    print("CV log_loss: ", score)
    
    #OOF save
    np.save(SAVEOOF + 'oof', y_pred)
    
    return score

## Postprocessing

In [28]:
# 特になし

## Submit

In [29]:
def Submit(confFitting, predictions, test):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

# Execute

In [40]:
def Exec(param):
    
    #Tester(True/False)
    Tester = True
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTarget)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [42]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, test, target, confFitting)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, target)
    
    # 課題提出
    #Submit(confFitting, predictions, test)
    
    return score


In [41]:
%%time
score= Exec(param_space)
print("score: " + str(score))

~~~~~~~~~~~~~~~~~~~~ SEED 42 ~~~~~~~~~~~~~~~~~~~~
==================== Fold 0 ====================
EPOCH: 000: | train_loss: 0.646: | valid_loss: 0.344
EPOCH: 001: | train_loss: 0.272: | valid_loss: 0.255
EPOCH: 002: | train_loss: 0.243: | valid_loss: 0.233
EPOCH: 003: | train_loss: 0.232: | valid_loss: 0.224
EPOCH: 004: | train_loss: 0.225: | valid_loss: 0.217
EPOCH: 005: | train_loss: 0.220: | valid_loss: 0.215
EPOCH: 006: | train_loss: 0.217: | valid_loss: 0.214
EPOCH: 007: | train_loss: 0.214: | valid_loss: 0.209
EPOCH: 008: | train_loss: 0.211: | valid_loss: 0.208
EPOCH: 009: | train_loss: 0.208: | valid_loss: 0.209
EPOCH: 010: | train_loss: 0.205: | valid_loss: 0.207
EPOCH: 011: | train_loss: 0.204: | valid_loss: 0.205
EPOCH: 012: | train_loss: 0.200: | valid_loss: 0.201
EPOCH: 013: | train_loss: 0.197: | valid_loss: 0.199
EPOCH: 014: | train_loss: 0.194: | valid_loss: 0.198
EPOCH: 015: | train_loss: 0.190: | valid_loss: 0.196
EPOCH: 016: | train_loss: 0.188: | valid_loss: 0.194


# Predict

In [32]:
def run_predict(confFitting, param, test, target, fold, seed):
    
    seed_everything(seed)
  
    #--------------------- PREDICTION---------------------
    x_test = test[confFitting["feature_cols"]].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.load_state_dict(torch.load(f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    
    return predictions


In [33]:
def run_k_fold_predict(confFitting, test, target, param, Tester, NFOLDS, seed):
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        pred_ = run_predict(confFitting, param, test, target, fold, seed)
        
        predictions += pred_ / NFOLDS
        
    return predictions

In [34]:
def SubmitPredict(confFitting, predictions, test, prefix):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}{prefix}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

In [35]:
def Predict(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        predictions_ = run_k_fold_predict(confFitting, test, target, param, Tester, NFOLDS, seed)
        predictions += predictions_ / len(SEED)
    
    # 課題提出
    prefix = "Pytorch"
    SubmitPredict(confFitting, predictions, test, prefix)
    
    return

In [36]:
%%time
#Predict(param_space)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


# Hyperparameter Tuning

In [37]:
#hyperopt
from hyperopt import fmin, tpe, hp, rand, Trials

In [38]:
def HOptExec(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, test, target, confFitting)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, train, target)
    
    # 課題提出
    #Submit(confFitting, predictions, test)
    
    return score

In [39]:
%%time

#param_space = {'hidden_size1': 512, 
#               'hidden_size2': 512, 
#               'dropOutRate1': 0.20393004966355735, 
#               'dropOutRate2': 0.39170486751620137,
#               'rankGauss_n_quantiles': 488.0393350201078,
#               'leakyReluSlope': hp.uniform('leakyReluSlope', 1e-3, 1e-1),
#              }
#
#trials = Trials()
#
#hopt = fmin(fn = HOptExec, 
#            space = param_space, 
#            algo = tpe.suggest, 
#            max_evals = 15, 
#            #timeout = 8.9 * 60 * 60, 
#            trials = trials, 
#           )
#
#print(hopt)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 µs
